In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PINNs_Chron import PINN3D, PINN3DSolver

# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)
tf.get_logger().setLevel('ERROR')

In [ ]:
# tf.config.list_physical_devices('GPU')

In [ ]:
%matplotlib inline

SIZE = 12
BIGGER_SIZE = 16

plt.rc('font', family='Arial', size=SIZE) # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize

In [ ]:
# 40 Ma model time is 110 Ma geo time
uplift = lambda t : np.where(t < 40, .6, .05)
tf_uplift = lambda t: tf.where(t < 40, .6, .05)
tf_uplift_inv = lambda t, u0, u1, t1: tf.where(t < t1, u0, u1)

In [ ]:
topo_ar = np.loadtxt('dabie_utm_km.xyz', dtype=DTYPE)
topo_x, topo_x_pos = np.unique(topo_ar[:, 0], return_inverse=True)
topo_y, topo_y_pos = np.unique(topo_ar[:, 1], return_inverse=True)
topo_pivot = np.zeros((len(topo_x), len(topo_y)), dtype=DTYPE)
topo_pivot[topo_x_pos, topo_y_pos] = topo_ar[:, 2]
topo_x_min, topo_x_max = topo_x.min(), topo_x.max()
topo_y_min, topo_y_max = topo_y.min(), topo_y.max()

In [ ]:
t_end = 150.
h0 = 30.
xl = topo_x.max() - topo_x.min()
yl = topo_y.max() - topo_y.min()
Tsea = 15.
Tbot = 600.
kappa = 25.
air_lapse = 5.

# Set number of data points
N_0 = 2000
N_b_b = 2000
N_b_s = 5000
N_b = N_b_b + N_b_s
N_r = 50000

# Set boundary
tmin = 0.
tmax = t_end
zmin = 0.
zmax = h0 + topo_ar[:, 2].max() * 6.
xmin = 0.
xmax = xl
ymin = 0.
ymax = yl

# Lower bounds
lb = tf.constant([tmin, xmin, ymin, zmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, xmax, ymax, zmax], dtype=DTYPE)

# Set random seed
tf.random.set_seed(128)

In [ ]:
import tensorflow_probability as tfp

def tf_h_fn_inv(t, x, y, amp0, t_decay):
    amp0 = tf.cast(amp0, dtype=DTYPE)
    one = tf.ones(1)
    amp = tf.where(t < t_decay, amp0, one + (amp0 - one) * (t_end - t)/ (t_end - t_decay))
    xy = tf.cast(tf.concat([x, y], axis=1), dtype=DTYPE)
    h_interp = tfp.math.batch_interp_regular_nd_grid(
        xy, [topo_x_min, topo_y_min,], [topo_x_max, topo_y_max], topo_pivot, axis=-2)
    return h0 + amp * tf.reshape(h_interp, tf.shape(x))

def tf_h_fn_init(x, y, amp0):
    zero = tf.zeros(1)
    one = tf.ones(1)
    return tf_h_fn_inv(zero, x, y, amp0, one)

def T_init(x, y, z, amp0):
    return Tbot - T_grad0(x, y, amp0) * z

# Define boundary condition
def T_surf(surf_z, sealevel_z=h0, lapse=air_lapse):
    elevation = surf_z - sealevel_z
    return tf.ones(tf.shape(surf_z), dtype=DTYPE) * Tsea - elevation * lapse

def T_bot(z):
    return tf.constant(Tbot, shape=z.shape, dtype=DTYPE)
    
# T_grad_inv = lambda t, x, y, amp0: (Tbot - T_surf(tf_h_fn_inv(t, x, y, amp0, t_decay=70.))) / tf_h_fn_inv(t, x, y, amp0, t_decay=70.)
T_grad_inv = lambda t, x, y, amp0, t_decay: (Tbot - T_surf(tf_h_fn_inv(t, x, y, amp0, t_decay))) / tf_h_fn_inv(t, x, y, amp0, t_decay)
T_grad0 = lambda x, y, amp0: (Tbot - T_surf(tf_h_fn_init(x, y, amp0))) / tf_h_fn_init(x, y, amp0)

In [ ]:
def generate_data(amp0, t_decay):    
    # Draw uniform sample points for initial boundary data
    t_0 = tf.ones((N_0, 1), dtype=DTYPE) * lb[0]
    x_0 = tf.random.uniform((N_0, 1), lb[1], ub[1], dtype=DTYPE)
    y_0 = tf.random.uniform((N_0, 1), lb[2], ub[2], dtype=DTYPE)
    z_0 = tf.random.uniform((N_0, 1), lb[3], tf_h_fn_init(x_0, y_0, amp0), dtype=DTYPE)
    X_0 = tf.concat([t_0, x_0, y_0, z_0], axis=1)
    # Evaluate intitial condition at z_0
    T_0 = T_init(x_0, y_0, z_0, amp0)

    # Boundary data
    mid_t = lb[0]+(ub[0]-lb[0])*.3 #to separate t space into two and sample each
    t_b0 = tf.random.uniform((N_b_b//2, 1), lb[0], mid_t, dtype=DTYPE)
    t_b1 = tf.random.uniform((N_b_b//2, 1), mid_t, ub[0], dtype=DTYPE)
    t_b = tf.concat([t_b0, t_b1], axis=0)

    x_b = tf.random.uniform((N_b_b, 1), lb[1], ub[1], dtype=DTYPE)
    y_b = tf.random.uniform((N_b_b, 1), lb[2], ub[2], dtype=DTYPE)
    z_b = lb[3] * tf.ones((N_b_b, 1), dtype=DTYPE)
    
    t_s0 = tf.random.uniform((N_b_s//2, 1), lb[0], mid_t, dtype=DTYPE)
    t_s1 = tf.random.uniform((N_b_s//2, 1), mid_t, ub[0], dtype=DTYPE)
    t_s = tf.concat([t_s0, t_s1], axis=0)

    x_s = tf.random.uniform((N_b_s, 1), lb[1], ub[1], dtype=DTYPE)
    y_s = tf.random.uniform((N_b_s, 1), lb[2], ub[2], dtype=DTYPE)
    z_s = tf.constant(tf_h_fn_inv(t_s, x_s, y_s, amp0, t_decay), dtype=DTYPE)

    t_bs = tf.concat([t_b, t_s], axis=0)
    x_bs = tf.concat([x_b, x_s], axis=0)
    y_bs = tf.concat([y_b, y_s], axis=0)
    z_bs = tf.concat([z_b, z_s], axis=0)
    X_bs = tf.concat([t_bs, x_bs, y_bs, z_bs], axis=1)

    # Evaluate boundary condition
    T_b = T_bot(z_b)
    T_s = T_surf(z_s)
    T_bs = tf.concat([T_b, T_s], axis=0)

    # Collect boundary and inital data in lists
    X_data = tf.concat([X_0, X_bs], axis=0)
    T_data = tf.concat([T_0, T_bs], axis=0)
    
    return X_data, T_data

In [ ]:
class PINNInv3D(PINN3D):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.u0 = tf.constant(1.)
        self.u1 = tf.constant(1.)
        self.t1 = tf.constant(25.)
        self.t2 = tf.constant(75.)
        self.amp0 = tf.constant(3.)
        
        self.transform = tf.keras.layers.Lambda(
            lambda x: x[:, 0:1] * (
                self.Tbot - T_grad_inv(x[:, 0:1], x[:, 1:2], x[:, 2:3], self.amp0, self.t2) * x[:, 3:4])   
                )

In [ ]:
from scipy.optimize import differential_evolution
from scipy.optimize import shgo
from scipy.optimize import direct
from scipy.optimize import dual_annealing
import os

class InvSolver(PINN3DSolver):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.age_loss_hist = []
        self.u1_hist = []
        self.t1_hist = []
        self.amp0_hist = []
        self.u0_hist = []
        self.t2_hist = [] #t2 is t_decay
        self.inv_it_hist = []
        self.inv_iter = 0
        #the following needs to be set up for optimization
        self.data_age = tf.constant([])
        self.data_method = []
        self.data_radi = []
        self.data_err = tf.constant([])
        self.data_x = []
        self.data_y = []
        self.savepath = 'saved_inv_model3d'
        
    def fun_u(self, t):
        return tf_uplift_inv(t, self.model.u0, self.model.u1, self.model.t1)
    
    def fun_h(self, t, x0, y0, amp0, t_amp0):
        return tf_h_fn_inv(t, [[x0]], [[y0]], amp0, t_amp0)[0]

    def fun_u_inv(self, t, x):
        u0, u1, t1, _, _ = x
        return tf_uplift_inv(t, tf.constant(u0, dtype=DTYPE),
                             tf.constant(u1, dtype=DTYPE),
                             tf.constant(t1, dtype=DTYPE))
    
    def get_tT(self, x0, y0, x):
        u0, u1, t1, t2, amp0 = x
        sample_t = tf.linspace(self.model.lb[0], self.model.ub[0], 100)
        u = self.fun_u_inv(sample_t, x)
        x_ar = tf.ones_like(u) * x0
        y_ar = tf.ones_like(u) * y0
        dpth = []
        for i in range(len(sample_t)):
            d = tfp.math.trapz(u[i:], x=sample_t[i:])
            dpth.append(d) # this is the depth relative to the present surface

        h_paleo = self.fun_h(sample_t, x0, y0, tf.constant(amp0, dtype=DTYPE), tf.constant(t2, dtype=DTYPE)) # paleo surface height
        h_present = self.fun_h(sample_t[-1], x0, y0, tf.constant(amp0, dtype=DTYPE), tf.constant(t2, dtype=DTYPE)) # present surface height
        depth = tf.stack(dpth) + h_paleo - h_present  # paleo depth
        
        z = self.fun_h(sample_t, x0, y0, self.model.amp0, self.model.t2) - depth
        sample_T = tf.reshape(self.model(tf.stack([sample_t, x_ar, y_ar, z], axis=1)), sample_t.shape)
        return sample_t, sample_T
            
    def age_loss(self, x):
        age_pred = []
        for hx, hy, hm, hr in zip(self.data_x, self.data_y, self.data_method, self.data_radi):
            sample_t, sample_T = self.get_tT(hx, hy, x)
            age_pred.append(self.pred_age(sample_t, sample_T, method=hm, grain_radius=hr))
            
        return tf.reduce_mean(tf.square((age_pred - self.data_age)/self.data_err))

    def solve_with_scipy(self, bounds,
                         samples=100, disp=True, maxiter=100, maxfun=None,
                         optimizer='shgo', sampling_method='simplicial',local_bias=False,
                         strategy='best2bin', mutation=(0.5, 1), recombination=0.7,
                         popsize=30, eps=1e-3):
        self.runpath = os.path.join(self.savepath, self.runname)
        if not(os.path.exists(self.runpath)):
            os.mkdir(self.runpath)
            
        def obj_fn(x):
            u0, u1, t1, t2, amp0 = x
            if t1 >= t2:
                return 99999.
            else:
                age_loss = self.age_loss(x)
                self.u0_hist.append(u0)
                self.u1_hist.append(u1)
                self.t1_hist.append(t1)
                self.amp0_hist.append(amp0)
                self.t2_hist.append(t2)
                self.inv_it_hist.append(self.inv_iter)
                self.current_age_loss = age_loss.numpy()
                self.age_loss_hist.append(self.current_age_loss)
            return age_loss

        def save_inv_history(file):
            np.savez(file, u0_hist=self.u0_hist, u1_hist=self.u1_hist, amp0_hist=self.amp0_hist,
                     t1_hist=self.t1_hist, t2_hist=self.t2_hist,
                     inv_it_hist=self.inv_it_hist, age_loss_hist=self.age_loss_hist
                     )
            
        def scipy_callback(*xf):
            self.inv_iter += 1
            x = xf[0]
            fun = obj_fn(x)
            runpath = self.get_runpath()
            print(('It {:6d} fun {:.4g}: run time = {:.4g} seconds;'
                   'u0 = {:.4g} u1 = {:.4g} t1 = {:.4g} t2 = {:.4g} amp0 = {:.4g}').format(
                   self.inv_iter, fun, time()-tic, x[0], x[1], x[2], x[3], x[4])
                 )
            save_inv_history(runpath + '/inv_solver_hist')
            self.save_history(runpath + '/solver_hist')
            with open(runpath + '/inv_log.txt', 'a') as log_f:
                log_f.write(f'it {self.inv_iter} - fun: {fun}; x: {x}\n')
                
            return self.current_loss < 1 and self.current_age_loss < .01

        tic = time()
        runpath = self.get_runpath()
        with open(runpath + '/inv_log.txt', 'a') as log_file:
            log_file.write(f'-- search begin -- bounds: {bounds}\n')

        if optimizer=='shgo':
            res = shgo(obj_fn, bounds, n=samples, options={'maxiter':maxiter, 'disp':disp},
                        sampling_method=sampling_method, callback=scipy_callback)
        elif optimizer=='DE':
            res = differential_evolution(obj_fn, bounds, strategy=strategy, init=sampling_method,
                   disp=disp, maxiter=maxiter, popsize=popsize, polish=local_bias,
                   mutation=mutation, recombination=recombination, callback=scipy_callback)
        elif optimizer=='direct':
            res = direct(obj_fn, bounds, maxiter=maxiter, maxfun=maxfun, eps=eps,
                          locally_biased=local_bias, callback=scipy_callback)
        elif optimizer=='DA':
            res = dual_annealing(obj_fn, bounds, maxiter=maxiter, maxfun=maxfun, callback=scipy_callback)
        else:
            raise Exception('optimizer is not available')

        with open(runpath + '/inv_log.txt', 'a') as log_file:
            log_file.write(f'-- search ends -- run time: {time() - tic:.0f}; optima: {res.x}\n')

        return res

In [ ]:
# Draw uniformly sampled collocation points
mid_t = lb[0]+(ub[0]-lb[0])*.3
t_r0 = tf.random.uniform((N_r//2, 1), lb[0], mid_t, dtype=DTYPE)
t_r1 = tf.random.uniform((N_r//2, 1), mid_t, ub[0], dtype=DTYPE)
t_r = tf.concat([t_r0, t_r1], axis=0)
# t_r = tf.random.uniform((N_r, 1), lb[0], ub[0], dtype=DTYPE)
x_r = tf.random.uniform((N_r, 1), lb[1], ub[1], dtype=DTYPE)
y_r = tf.random.uniform((N_r, 1), lb[2], ub[2], dtype=DTYPE)
z_r = tf.random.uniform((N_r, 1), lb[3], tf_h_fn_init(x_r, y_r, 1.), dtype=DTYPE)
X_r = tf.concat([t_r, x_r, y_r, z_r], axis=1)

In [ ]:
def read_ages(txt, DYTPE='float32'):
    ar = np.loadtxt(txt, dtype='str')
    x = ar[:, 0].astype(DTYPE)
    y = ar[:, 1].astype(DTYPE)
    ages = tf.constant(ar[:, 2].astype(DTYPE))
    errs = tf.constant(ar[:, 3].astype(DTYPE))
    radi = ar[:, 4].astype(DTYPE)
    methods = ar[:, 5]
    return x, y, ages, errs, radi, methods

In [ ]:
def l_rate(step, initial_rate=1e-3, decay_rate=.9, decay_steps=1000):
    return initial_rate * decay_rate ** (step / decay_steps)
    
# x = np.linspace(0, 500000, 1000)
# plt.plot(x,  l_rate(x, decay_steps=10000))
# plt.yscale('log')

In [ ]:
# Initialize model
model = PINNInv3D(lb, ub, num_hidden_layers=16, num_neurons_per_layer=20)
model.build(input_shape=(None, 4))

# lr = 1e-3
# lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([20000],[1e-2, 1e-3])
lr = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, decay_steps=10000, decay_rate=0.9)
optim = tf.keras.optimizers.Adam(learning_rate=lr)

# Initilize PINN solver
solver = InvSolver(model, X_r)
solver.savepath = 'saved_inv_model3d'

solver.data_x, solver.data_y, solver.data_age, solver.data_err, solver.data_radi, solver.data_method = read_ages('dabie_ages.txt')

In [ ]:
def shrink(b0, x, frac=.9):
    d = (np.max(b0) - np.min(b0)) * frac * .5
    b1l = x - d
    b1u = x + d
    dl = b1l - np.min(b0)
    du = np.max(b0) - b1u
    if dl < 0:
        b1l -= dl
        b1u -= dl
    elif du < 0:
        b1l += du
        b1u += du

    return (b1l, b1u) 
    
def new_bounds(bounds, optima, optima_0, frac=.9):
    new_bounds = []
    for b0, x, op0 in zip(bounds, optima, optima_0):
        r = np.diff(b0)[0]
        if np.abs((x - op0) / r) < .05:
            b1 = shrink(b0, x, frac=frac)
        else:
            b1 = b0
            
        new_bounds.append(b1)
        
    return new_bounds

def zoom_in(bounds, optima, optima_0):
    new_bounds = []
    for b0, x, op0 in zip(bounds, optima, optima_0):
        r = np.diff(b0)[0]
        if np.abs((x - op0) / r) < .05:
            d = np.abs(np.array(b0) - x).min()
            b1 = (x - d, x + d)
        else:
            b1 = b0
            
        new_bounds.append(b1)

    return new_bounds

In [ ]:
def new_adam_n(adam_n, up_by=1.5, max=10000):
    return min(max, np.int32(adam_n * up_by))

In [ ]:
from time import time
# Start timer
tic = time()

bounds = [(0., 1.), (0., 1.), (0., 50.), (50., 100.), (0., 6.)]
ops = [np.mean(a) for a in bounds]
solver.model.u0, solver.model.u1, solver.model.t1, solver.model.t2, solver.model.amp0 = tf.constant(ops, dtype=DTYPE)

maxiter, maxfun = 30 + 2, 1000
runpath = solver.get_runpath()
np.save(runpath  + '/inv_X_r', X_r)

adam_n = 100
for i in range(50):
    X_data, T_data = generate_data(solver.model.amp0, solver.model.t2)
    solver.solve_with_Adam(optim, X_data, T_data, N=adam_n, echofreq=adam_n, savefreq=-1)
    res = solver.solve_with_scipy(bounds, maxiter=maxiter, maxfun=maxfun,
                                  optimizer='direct', eps=1e-1, local_bias=False
                                  )
    tf.saved_model.save(model, os.path.join(runpath, 'model' + str(i)))
    bounds = new_bounds(bounds, res.x, ops, frac=.9)
    # bounds = zoom_in(bounds, res.x, ops)
    ops = res.x
    adam_n = new_adam_n(adam_n, up_by=1.2, max=20000)
    solver.model.u0, solver.model.u1, solver.model.t1, solver.model.t2, solver.model.amp0 = tf.constant(ops, dtype=DTYPE)

# Print computation time
print('\nComputation time: {} seconds'.format(time() - tic));